In [22]:
def var_exist(ds,varin):
    for var in ds:
        if var==varin:
            return True
    return False

import xarray as xr
import numpy as np
from glob import glob
import os

adir_usv = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/*.nc' 

#list names of variables to keep
list_var = ['time','lat','lon','SOG_MEAN','COG_MEAN','HDG_MEAN','ROLL_MEAN','PITCH_MEAN',
            'UWND_MEAN','VWND_MEAN','WWND_MEAN','GUST_WND_MEAN','TEMP_AIR_MEAN','RH_MEAN','BARO_PRES_MEAN',
            'PAR_AIR_MEAN','TEMP_CTD_MEAN','SAL_CTD_MEAN','TEMP_RBR_MEAN','SAL_RBR_MEAN',
            'TEMP_O2_RBR_MEAN','CDOM_MEAN','CHLOR_MEAN','CHLOR_WETLABS_MEAN','WIND_MEASUREMENT_HEIGHT_MEAN']
#list names of variables to swap to common names
swapvar = {'TEMP_SBE37_MEAN':'TEMP_CTD_MEAN',
           'SAL_SBE37_MEAN':'SAL_CTD_MEAN',
           'SAL_MEAN':'SAL_CTD_MEAN',
           'TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN',
           'TEMP_CTD_RBR_MEAN':'TEMP_RBR_MEAN',
           'CHLOR_RBR_MEAN':'CHLOR_MEAN',
           'SOG':'SOG_MEAN','SOG_FILTERED_MEAN':'SOG_MEAN',
           'COG':'COG_MEAN','COG_FILTERED_MEAN':'COG_MEAN',
           'HDG':'HDG_MEAN','HDG_FILTERED_MEAN':'HGD_MEAN',
           'ROLL_FILTERED_MEAN':'ROLL_MEAN','ROLL_FILTERD_MEAN':'ROLL_MEAN',
           'PITCH_FILTERED_MEAN':'PITCH_MEAN','PITCH':'PITCH_MEAN',
           'WIND_HEIGHT_MEAN':'WIND_MEASUREMENT_HEIGHT_MEAN'}

#get list of all filenames in directory
files = [x for x in glob(adir_usv)]
print('number of file:',len(files))

#go through each file, read in, normalize and put in dictionary with datasets
for ifile,file in enumerate(files):
    if not ifile==31:
        continue
    #print(file)
    ds = xr.open_dataset(file)
    ds.close()
    ds_save = ds.copy(deep=True)
    if any(v=='latitude' for v in ds.coords.keys()):
        ds = ds.rename({'latitude':'lat','longitude':'lon'})
    if any(v=='latitude' for v in ds.dims.keys()):
        ds = ds.rename({'latitude':'lat','longitude':'lon'})
    if any(v=='latitude' for v in ds):
        ds = ds.rename({'latitude':'lat','longitude':'lon'})
    if any(v=='trajectory' for v in ds.dims.keys()):
        ds = ds.isel(trajectory=0)
#    for v in ds.dims.keys():
    if any(v=='obs' for v in ds.dims.keys()):
        ds = ds.swap_dims({'obs':'time'})
    if any(v=='row' for v in ds.dims.keys()):
        ds = ds.swap_dims({'row':'time'})
    #remove any duplicates in time, keep only first value
    _, index = np.unique(ds['time'], return_index=True)
    ds=ds.isel(time=index)
    #renames some common variables to uniform name, drop variables not on list above
    if any(var=='wind_speed' for var in ds):
        #print(ds.wind_speed.attrs)
        #saildrone using meterological wind (blowind from) in early cruises
        #since noaa is calculating from uwnd and vwnd I went to 2019 arctic cruise 1037 and double checked values
        ds['UWND_MEAN']=-ds.wind_speed*np.sin(np.deg2rad(ds.wind_dir))
        ds['VWND_MEAN']=-ds.wind_speed*np.cos(np.deg2rad(ds.wind_dir))
#            ds.UWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
#            ds.VWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
        ds.UWND_MEAN.attrs = {'standard_name': 'eastward_wind', 'long_name': 'Eastward wind speed',
                              'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}
        ds.VWND_MEAN.attrs = {'standard_name': 'northward_wind', 'long_name': 'Northward wind speed',
                              'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}

    # SWAP VARIABLE NAMES WHERE POSSIBLE
    for var in ds:
        if swapvar.get(var): 
            if not var_exist(ds,swapvar.get(var)):
                ds = ds.rename({var:swapvar.get(var)})

    #check that there is a TEMP_CTD_MEAN, if not & temp_rbr_mean there, change it to temp_ctd_mean
    if not any(var=='TEMP_CTD_MEAN' for var in ds):
        if any(var=='TEMP_RBR_MEAN' for var in ds):
            ds = ds.rename({'TEMP_RBR_MEAN':'TEMP_CTD_MEAN'})
        if any(var=='TEMP_CTD_RBR_MEAN' for var in ds):
            ds = ds.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN'})
    if not any(var=='SAL_CTD_MEAN' for var in ds):
        if any(var=='SAL_RBR_MEAN' for var in ds):
            ds = ds.rename({'SAL_RBR_MEAN':'SAL_CTD_MEAN'})

    # DROP VARIABLES NOT LISTED IN LIST_VAR
    for var in ds:
        var2 = var
        if not any(vv==var2 for vv in list_var):
            ds = ds.drop(var2)               



number of file: 44
SOG
COG
HDG
PITCH
SAL_MEAN
TEMP_O2_RBR_MEAN


In [23]:
ds

<xarray.Dataset>
Dimensions:         (time: 177279)
Coordinates:
    trajectory      (time) object ...
    lat             (time) float64 ...
    lon             (time) float64 ...
  * time            (time) datetime64[ns] 2018-07-17T18:00:00 ... 2018-11-27T19:59:00
Data variables:
    SOG_MEAN        (time) float64 ...
    COG_MEAN        (time) float64 ...
    HDG_MEAN        (time) float64 ...
    PITCH_MEAN      (time) float64 ...
    BARO_PRES_MEAN  (time) float64 ...
    TEMP_AIR_MEAN   (time) float64 ...
    RH_MEAN         (time) float64 ...
    PAR_AIR_MEAN    (time) float64 ...
    UWND_MEAN       (time) float64 3.97 3.96 4.79 4.96 ... -0.3 -0.31 -0.42
    VWND_MEAN       (time) float64 -6.95 -6.46 -6.93 -7.55 ... 5.6 5.35 5.62
    WWND_MEAN       (time) float64 ...
    GUST_WND_MEAN   (time) float64 ...
    TEMP_CTD_MEAN   (time) float64 ...
    SAL_CTD_MEAN    (time) float64 ...
    CHLOR_MEAN      (time) float64 ...
Attributes:
    area:                      unassigned
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    creator_url:               http://saildrone.com/
    data_mode:                 realtime
    date_created:              2018-10-06T00:30:01.915026Z
    description:               Saildrone NetCDF Format
    drone_id:                  1028
    Easternmost_Easting:       -120.5020928
    featureType:               Trajectory
    geospatial_lat_max:        40.0555744
    geospatial_lat_min:        33.8146944
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -120.5020928
    geospatial_lon_min:        -124.915008
    geospatial_lon_units:      degrees_east
    history:                   FERRET V7.504  21-Jul-19\n2020-05-07T16:30:14Z...
    id:                        8708
    infoUrl:                   http://saildrone.com/
    institution:               Saildrone
    interval:                  60.0
    keywords:                  active, air, air_pressure, air_temperature, an...
    keywords_vocabulary:       GCMD Science Keywords
    license:                   The data may be used and redistributed for fre...
    naming_authority:          com.saildrone
    ncei_template_version:     NCEI_NetCDF_Trajectory_Template_v2.0
    netcdf_version:            4.4.1.1
    Northernmost_Northing:     40.0555744
    platform:                  Saildrone
    project:                   west_coast_survey_2018
    sourceUrl:                 (local files)
    Southernmost_Northing:     33.8146944
    standard_name_vocabulary:  CF Standard Name Table v29
    subsetVariables:           trajectory,BARO_PRES_serial_number,AT_RH_seria...
    summary:                   Saildrone Network Common Data Format (NetCDF) ...
    testOutOfDate:             now-1day
    time_coverage_duration:    PT1H59M59S
    time_coverage_end:         2018-11-27T19:59:00Z
    time_coverage_resolution:  PT1M
    time_coverage_start:       2018-07-17T18:00:00Z
    title:                     Saildrone West Coast Survey 2018
    uuid:                      3e4ce677-18db-4ba8-9c73-f2c1f3ad15ce
    Westernmost_Easting:       -124.915008

In [20]:
    # SWAP VARIABLE NAMES WHERE POSSIBLE
    for var in ds:
        if swapvar.get(var): 
            print(var)
            if not var_exist(ds,swapvar.get(var)):
                ds = ds.rename({var:swapvar.get(var)})


In [21]:
ds

<xarray.Dataset>
Dimensions:                        (row: 177279)
Coordinates:
    trajectory                     (row) object '1024.0' '1024.0' ... '1024.0'
    latitude                       (row) float64 38.23 38.23 38.23 ... 37.8 37.8
    longitude                      (row) float64 -123.9 -123.9 ... -122.6 -122.6
    time                           (row) datetime64[ns] 2018-07-17T18:00:00 ... 2018-11-27T19:59:00
Dimensions without coordinates: row
Data variables:
    SOG_MEAN                       (row) float64 1.228 1.296 ... 0.928 0.701
    COG_MEAN                       (row) float64 276.6 279.1 278.0 ... 71.2 57.9
    HDG_MEAN                       (row) float64 279.6 286.8 276.2 ... 80.7 82.5
    HDG_WING                       (row) float64 309.2 309.3 ... 155.2 161.9
    ROLL                           (row) float64 8.0 7.3 7.7 8.3 ... 4.1 4.7 4.7
    PITCH_MEAN                     (row) float64 2.7 2.9 2.6 2.4 ... 2.0 2.0 2.2
    WING_ANGLE                     (row) float64 29.7 22.5 27.5 ... 74.5 79.4
    BARO_PRES_MEAN                 (row) float64 1.018e+03 ... 1.019e+03
    BARO_PRES_STDDEV               (row) float64 0.07 0.07 0.07 ... 0.12 0.13
    TEMP_AIR_MEAN                  (row) float64 13.89 13.89 ... 14.23 14.24
    TEMP_AIR_STDDEV                (row) float64 0.02 0.02 0.04 ... 0.01 0.01
    RH_MEAN                        (row) float64 95.24 95.5 ... 95.22 95.16
    RH_STDDEV                      (row) float64 0.2 0.18 0.31 ... 0.04 0.05
    PAR_AIR_MEAN                   (row) float64 521.0 529.0 ... 634.0 616.0
    PAR_AIR_STDDEV                 (row) float64 6.0 4.0 5.0 ... 13.0 9.0 7.0
    TEMP_IR_UNCOR_MEAN             (row) float64 15.42 nan nan ... 14.07 14.06
    TEMP_IR_UNCOR_STDDEV           (row) float64 0.04 nan nan ... 0.02 0.02 0.02
    UWND_MEAN                      (row) float64 3.97 3.96 4.79 ... -0.31 -0.42
    UWND_STDDEV                    (row) float64 0.49 0.51 0.75 ... 0.37 0.42
    VWND_MEAN                      (row) float64 -6.95 -6.46 -6.93 ... 5.35 5.62
    VWND_STDDEV                    (row) float64 0.7 0.63 0.68 ... 0.48 0.42
    WWND_MEAN                      (row) float64 0.06 -0.03 0.05 ... -0.16 -0.19
    WWND_STDDEV                    (row) float64 0.37 0.4 0.42 ... 0.33 0.36
    GUST_WND_MEAN                  (row) float64 9.02 8.73 9.43 ... 6.12 6.14
    GUST_WND_STDDEV                (row) float64 0.57 0.44 0.59 ... 0.3 0.4 0.3
    TEMP_CTD_MEAN                  (row) float64 nan 15.2 15.2 ... 13.81 13.81
    TEMP_CTD_STDDEV                (row) float64 nan 0.005 0.006 ... 0.004 0.007
    COND_MEAN                      (row) float64 nan 40.98 40.99 ... 37.28 37.3
    COND_STDDEV                    (row) float64 nan 0.001 0.001 ... 0.004 0.012
    SAL_CTD_MEAN                   (row) float64 nan 33.07 33.08 ... 30.85 30.86
    SAL_STDDEV                     (row) float64 nan 0.004 0.005 ... 0.005 0.013
    O2_RBR_CONC_MEAN               (row) float64 nan 253.3 253.3 ... 231.1 231.1
    O2_RBR_CONC_STDDEV             (row) float64 nan 0.21 0.14 ... 0.19 0.15
    O2_RBR_SAT_MEAN                (row) float64 nan 99.03 98.99 ... 86.6 86.61
    O2_RBR_SAT_STDDEV              (row) float64 nan 0.08 0.06 ... 0.08 0.06
    TEMP_O2_MEAN                   (row) float64 nan 15.21 15.21 ... 13.82 13.82
    TEMP_O2_RBR_STDDEV             (row) float64 nan 0.002 0.003 ... 0.003 0.002
    O2_AANDERAA_CONC_UNCOR_MEAN    (row) float64 nan nan nan nan ... nan nan nan
    O2_AANDERAA_CONC_UNCOR_STDDEV  (row) float64 nan nan nan nan ... nan nan nan
    O2_AANDERAA_SAT_MEAN           (row) float64 nan nan nan nan ... nan nan nan
    O2_AANDERAA_SAT_STDDEV         (row) float64 nan nan nan nan ... nan nan nan
    TEMP_O2_AANDERAA_MEAN          (row) float64 nan nan nan nan ... nan nan nan
    TEMP_O2_AANDERAA_STDDEV        (row) float64 nan nan nan nan ... nan nan nan
    CHLOR_MEAN                     (row) float64 nan 0.6168 ... 4.182 4.166
    CHLOR_STDDEV                   (row) flo